#### Part1: Extracting data
###### It will extract details- city, station, date_column from inside file or filename
###### It will then extract data and save it as df.
###### Summary table is created and saved
###### Hourly average plots are created for selected polluatnst by user.

In [ ]:
import pandas as pd
import os
import re

# Parameter-Unit Dictionery (Manually Added)
units_dict = {
    "PM2.5": "µg/m³",
    "PM10": "µg/m³",
    "NO": "µg/m³",
    "NO2": "µg/m³",
    "NOx": "ppb",
    "NH3": "µg/m³",
    "SO2": "µg/m³",
    "CO": "mg/m³",
    "Ozone": "µg/m³",
    "Benzene": "µg/m³",
    "Toluene": "µg/m³",
    "Xylene": "µg/m³",
    "O Xylene": "µg/m³",
    "Eth-Benzene": "µg/m³",
    "MP-Xylene": "µg/m³",
    "AT": "°C",
    "Temp": "°C",
    "RH": "%",
    "WS": "m/s",
    "WD": "deg",
    "RF": "mm",
    "TOT-RF": "mm",
    "SR": "W/m²",
    "BP": "mmHg",
    "VWS": "m/s"
}

file_path = "C:/Users/shafi/Music/CPCB/City_Bangalore/Raw_data_1Day_2024_site_162_BTM_Layout_Bengaluru_CPCB_1Day.csv"  #file path
file_ext = os.path.splitext(file_path)[-1].lower()

if file_ext == ".xlsx":
    df = pd.read_excel(file_path, header=None)
    
  
    station_name, city_name = None, None #Extractig station, and city, before processing data
    for i in range(15):  # Checking fir..st 15 rows for metadata
        if isinstance(df.iloc[i, 0], str):
            if "Station" in df.iloc[i, 0]:
                station_name = df.iloc[i, 1].split(",")[0]  # Merged cell value, take until first comma
            if "City" in df.iloc[i, 0]:
                city_name = df.iloc[i, 1]  # Mergd cell value
    
    # Identifying the header row (row where 'From Date' appears in column 1)
    header_row = df[df.iloc[:, 0] == "From Date"].index[0]
    df.columns = df.iloc[header_row]
    df = df[header_row + 1:].reset_index(drop=True)
    
    # If station and city not found, extract from filename
    if not station_name or not city_name:
        filename = os.path.basename(file_path)
        parts = filename.split("_")
        if "site" in parts:
            site_index = parts.index("site")
            if site_index + 2 < len(parts):
                station_name = parts[site_index + 2]  # Aftr numerical number
        if "CPCB" in parts:
            cpcb_index = parts.index("CPCB")
            city_name = parts[cpcb_index - 1]

    # Identify date column (keeping only 'From Date')
    date_column = "From Date"
    df = df[["From Date"] + [col for col in df.columns if col not in ["From Date", "To Date"]]]
    
    # Extract parameter names without units
    df.columns = [re.sub(r"\s*\(.*?\)", "", str(col)) for col in df.columns]
    parameters = [col for col in df.columns if col != "From Date"]
    num_parameters = len(parameters)

elif file_ext == ".csv":
    
    df = pd.read_csv(file_path)
    
    
    filename = os.path.basename(file_path) # Extract station and city name from filename
    parts = filename.split("_")
    if "site" in parts:
        site_index = parts.index("site")
        if site_index + 2 < len(parts):
            station_name = parts[site_index + 2]  # Aftr numerical number
    if "CPCB" in parts:
        cpcb_index = parts.index("CPCB")
        city_name = parts[cpcb_index - 1]
    
    date_column = "Timestamp"
    
    # Extract parameter names without units
    df.columns = [re.sub(r"\s*\(.*?\)", "", str(col)) for col in df.columns]
    parameters = [col for col in df.columns if col != "Timestamp"]
    num_parameters = len(parameters)

df.replace(["None", "NA", "Na"], pd.NA, inplace=True)

df = df.dropna(axis=1, how='all')

print("Station Name:", station_name)
print("City Name:", city_name)
print("Date Column:", date_column)
print("Number of Parameters:", num_parameters)
print("Parameters and Units:", {param: units_dict.get(param, "Unknown") for param in parameters})

print(df.head())


In [ ]:
import pandas as pd
import os

# Dropping duplicate columns
df = df.loc[:, ~df.columns.duplicated()]

# Ensure the correct date column name is set
#date_column = "From Date"  
df.loc[:, date_column] = pd.to_datetime(df[date_column], errors="coerce",  dayfirst=True)  # Fix Warning


df = df.dropna(subset=[date_column])

numeric_columns = df.columns.difference([date_column]) # Identify numeric columns, excluding the date colun


# Convert all numeric columns to float
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")

# Drop rows where all numeric values are NaN
df = df.dropna(subset=numeric_columns, how="all")

# Ensure there are still valid numeric columns
if df[numeric_columns].shape[1] == 0:
    raise ValueError("No numeric data available for aggregation.")

# Group data by date and calculate daily mean for each parameter
daily_mean = df.groupby(df[date_column].dt.date).mean(numeric_only=True)


metrics = ["Mean", "Median", "Std. Dev", "Min", "Max"]
summary_data = {"Metrics": metrics}

valid_parameters = [param for param in parameters if param in df.columns]

for param in valid_parameters:
    final_mean = daily_mean[param].mean()
    final_median = df[param].median()
    final_std = daily_mean[param].std()
    final_min = df[param].min()
    final_max = df[param].max()
    
    summary_data[param] = [final_mean, final_median, final_std, final_min, final_max]

summary_df = pd.DataFrame(summary_data)

# Insert unit row below headers
unit_row = ["Units"] + [units_dict.get(param, "Unknown") for param in valid_parameters]
unit_df = pd.DataFrame([unit_row], columns=["Metrics"] + valid_parameters)

# 🔹 Fix Column Order Mismatch Issue Before `pd.concat`
summary_df = summary_df.reindex(columns=["Metrics"] + valid_parameters, fill_value="Unknown")

# Concatenate the unit row and metrics column with the summary DataFrame
summary_df = pd.concat([unit_df, summary_df], ignore_index=True)
summary_df["Metrics"] = ["Units"] + metrics  # Ensure the first column is correct


output_filename = f"{station_name}_{city_name}_SummaryTable.xlsx"
output_path = os.path.join(os.getcwd(), output_filename)

with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    summary_df.to_excel(writer, index=False, header=False, startrow=1)
    worksheet = writer.sheets['Sheet1']
    
    # Write the actual headers manually (pollutant names)
    for col_num, value in enumerate(summary_df.columns):
        worksheet.write(0, col_num, value)

print(f"Summary file saved as: {output_filename}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import tkinter as tk
from tkinter import simpledialog

# Ensure date_time column is in datetime format
df[date_column] = pd.to_datetime(df[date_column])

# Extract date and hour separately
df["Date"] = df[date_column].dt.date  # Extract date (ignoring time)
df["Hour"] = df[date_column].dt.hour  # Extract hour

# Check if hourly data exists by verifying if the hour changes within at least one day
hourly_data_available = df.groupby("Date")["Hour"].nunique().max() > 1

if not hourly_data_available:
    print("Hourly data is not available. The dataset contains only daily values.")
else:
    # Compute hourly average across all days
    hourly_avg = df.groupby("Hour").mean()

    # Use tkinter to allow user selection of pollutants
    root = tk.Tk()
    root.withdraw()  # Hide root window

    # Get available pollutants
    pollutants_available = [param for param in parameters if param in df.columns]

    if not pollutants_available:
        print("No pollutants available for plotting.")
    else:
        # Ask user to select pollutants
        selected_pollutants = simpledialog.askstring(
            "Select Pollutants",
            f"Available pollutants: {', '.join(pollutants_available)}\nEnter pollutant names separated by commas:"
        )

        if selected_pollutants:
            selected_pollutants = [p.strip() for p in selected_pollutants.split(",") if p.strip() in pollutants_available]
            for pollutant in selected_pollutants:
                plt.figure(figsize=(10, 5))

                # Plot hourly average
                plt.plot(hourly_avg.index, hourly_avg[pollutant], marker="o", linestyle="-", color="b")

                # Formatting
                plt.xticks(range(24),fontsize=12)
                plt.yticks(fontsize=12)
                plt.xlabel("Hour of the day", fontsize=14)
                plt.ylabel(f"{pollutant} ({units_dict.get(pollutant, 'Unknown')})", fontsize=14)
                plt.title(f"Hourly Average Variation of {pollutant}", fontsize=16)
                plt.grid(True, linestyle="--", alpha=0.7)

                
                output_filename = f"{station_name}_{city_name}_{pollutant}_Hourly_Average_Variations.jpg"
                output_path = os.path.join(os.getcwd(), output_filename)
                plt.savefig(output_path, dpi=300, bbox_inches="tight")
                plt.close()
                
                print(f"Plot saved: {output_filename}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tkinter as tk
from tkinter import simpledialog

# Ensure date_time column is in datetime format
df[date_column] = pd.to_datetime(df[date_column])

# Extract date and hour separately
df["Date"] = df[date_column].dt.date  # Extract date (ignoring time)
df["Hour"] = df[date_column].dt.hour  # Extract hour

# Check if hourly data exists
hourly_data_available = df.groupby("Date")["Hour"].nunique().max() > 1

# Use Tkinter for user input
root = tk.Tk()
root.withdraw()  # Hide root window

# Ask user whether they want distribution on hourly data or whole dataset
plot_scope = simpledialog.askstring(
    "Select Data Scope",
    "Do you want distribution plots based on hourly trends ('hourly') or for the entire dataset as a whole ('whole')?"
).strip().lower()

if plot_scope not in ["hourly", "whole"]:
    print("Invalid input! Defaulting to 'whole'.")
    plot_scope = "whole"

# Ask user whether they want a box plot or violin plot
plot_type = simpledialog.askstring(
    "Select Plot Type",
    "Enter 'box' for Box Plot or 'violin' for Violin Plot:"
).strip().lower()

if plot_type not in ["box", "violin"]:
    print("Invalid choice! Defaulting to box plot.")
    plot_type = "box"

# Get available pollutants
pollutants_available = [param for param in parameters if param in df.columns]

if not pollutants_available:
    print("No pollutants available for plotting.")
else:
    if plot_scope == "whole":
        # User wants distribution plots for whole dataset
        selected_pollutants = simpledialog.askstring(
            "Select Pollutants",
            f"Available pollutants: {', '.join(pollutants_available)}\nEnter 'all' to plot all pollutants, or enter pollutant names separated by commas:"
        )

        if selected_pollutants:
            if selected_pollutants.strip().lower() == "all":
                selected_pollutants = pollutants_available  # Select all pollutants

                # Plot all pollutants in one figure (each pollutant as separate box/violin)
                plt.figure(figsize=(12, 6))

                # Convert data into long format for Seaborn
                df_long = df[selected_pollutants].melt(var_name="Pollutant", value_name="Concentration")

                # Create box plot or violin plot
                if plot_type == "box":
                    sns.boxplot(x="Pollutant", y="Concentration", data=df_long, palette="Blues")
                    plot_title = "Box Plot of All Pollutants"
                else:
                    sns.violinplot(x="Pollutant", y="Concentration", data=df_long, palette="Blues", inner="quartile")
                    plot_title = "Violin Plot of All Pollutants"

                # Formatting
                plt.xticks(rotation=45, fontsize=12)
                plt.yticks(fontsize=12)
                plt.xlabel("Pollutant", fontsize=14)
                plt.ylabel("")
                plt.title(plot_title, fontsize=16)
                plt.grid(True, linestyle="--", alpha=0.7)

                
                output_filename = f"{station_name}_{city_name}_All_Pollutants_Whole_{plot_type.capitalize()}_Plot.jpg"
                output_path = os.path.join(os.getcwd(), output_filename)
                plt.savefig(output_path, dpi=300, bbox_inches="tight")
                plt.close()

                print(f"Plot saved: {output_filename}")

            else:
                selected_pollutants = [p.strip() for p in selected_pollutants.split(",") if p.strip() in pollutants_available]

                for pollutant in selected_pollutants:
                    plt.figure(figsize=(12, 6))

                    
                    if plot_type == "box":
                        sns.boxplot(y=df[pollutant], palette="Blues")
                        plot_title = f"Box Plot of {pollutant} "
                    else:
                        sns.violinplot(y=df[pollutant], palette="Blues", inner="quartile")
                        plot_title = f"Violin Plot of {pollutant}"

                    # Formatting
                    plt.xticks([])
                    plt.yticks(fontsize=12)
                    plt.xlabel("")
                    plt.ylabel(f"{pollutant} ({units_dict.get(pollutant, 'Unknown')})", fontsize=14)
                    plt.title(plot_title, fontsize=16)
                    plt.grid(True, linestyle="--", alpha=0.7)

            
                    output_filename = f"{station_name}_{city_name}_{pollutant}_Whole_{plot_type.capitalize()}_Plot.jpg"
                    output_path = os.path.join(os.getcwd(), output_filename)
                    plt.savefig(output_path, dpi=300, bbox_inches="tight")
                    plt.close()

                    print(f"Plot saved: {output_filename}")
                    
    if plot_scope == "hourly":
        # Asking user to select pollutants for hourly data
        selected_pollutants = simpledialog.askstring(
            "Select Pollutants",
            f"Available pollutants: {', '.join(pollutants_available)}\nEnter pollutant names separated by commas:"
        )

        if selected_pollutants:
            selected_pollutants = [p.strip() for p in selected_pollutants.split(",") if p.strip() in pollutants_available]

            if not selected_pollutants:
                print("No valid pollutants selected. Exiting.")
            else:
                for pollutant in selected_pollutants:
                    plt.figure(figsize=(12, 6))

                    # Plot box/violin plot on hourly data
                    if plot_type == "box":
                        sns.boxplot(x=df["Hour"], y=df[pollutant], palette="Blues")
                        plot_title = f"Hourly Box Plot of {pollutant}"
                    else:
                        sns.violinplot(x=df["Hour"], y=df[pollutant], palette="Blues", inner="quartile")
                        plot_title = f"Hourly Violin Plot of {pollutant}"

                    # Formatting
                    plt.xticks(fontsize=12)
                    plt.yticks(fontsize=12)
                    plt.xlabel("Hour of the Day", fontsize=14)
                    plt.ylabel(f"{pollutant} ({units_dict.get(pollutant, 'Unknown')})", fontsize=14)
                    plt.title(plot_title, fontsize=16)
                    plt.grid(True, linestyle="--", alpha=0.7)

                
                    output_filename = f"{station_name}_{city_name}_{pollutant}_Hourly_{plot_type.capitalize()}_Plot.jpg"
                    output_path = os.path.join(os.getcwd(), output_filename)
                    plt.savefig(output_path, dpi=300, bbox_inches="tight")
                    plt.close()

                    print(f"Plot saved: {output_filename}")

In [ ]:
import os
import tkinter as tk
from tkinter import simpledialog
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


# Tkinter for user input
root = tk.Tk()
root.withdraw()  # Hide root window

df.set_index(date_column, inplace=True)

# Asking user if they want a line plot
plot_line = simpledialog.askstring(
    "Line Plot Selection",
    "Do you want a line plot? (yes/no)"
).strip().lower()

if plot_line == "yes":
    # Ask user to select pollutants
    selected_pollutants = simpledialog.askstring(
        "Select Pollutants",
        f"Available pollutants: {', '.join(pollutants_available)}\nEnter 'all' to plot all pollutants, or enter pollutant names separated by commas:"
    )
    
    if selected_pollutants:
        if selected_pollutants.strip().lower() == "all":
            plt.figure(figsize=(12, 6))
            
            for pollutant in pollutants_available:
                plt.plot(df.index, df[pollutant], marker='o', linestyle='-', label=pollutant)
            
            plt.xlabel("Time", fontsize=14)
            plt.ylabel("")  # Remove y-axis label for all pollutants
            plt.title("Time Series of All Pollutants", fontsize=16)
            plt.legend(title="Pollutants", fontsize=12)
            plt.grid(True, linestyle="--", alpha=0.7)

            # Format x-axis correctly
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%d-%m-%Y"))  # Day-Month-Year
            plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())  # Auto-pick best date intervals
            plt.xticks(rotation=45)

            # Save plot as JPG
            
            output_filename = f"{station_name}_{city_name}_All_Pollutants_Line_Plot.jpg"
            output_path = os.path.join(os.getcwd(), output_filename)
            plt.savefig(output_path, dpi=300, bbox_inches="tight")
            plt.close()

            print(f"Plot saved: {output_filename}")
        else:
            selected_pollutants = [p.strip() for p in selected_pollutants.split(",") if p.strip() in pollutants_available]
            
            for pollutant in selected_pollutants:
                plt.figure(figsize=(12, 6))
                plt.plot(df.index, df[pollutant], marker='o', linestyle='-', color='b')
                
                plt.xlabel("Time", fontsize=14)
                plt.ylabel(f"{pollutant} ({units_dict.get(pollutant, 'Unknown')})", fontsize=14)
                plt.title(f"Time Series of {pollutant}", fontsize=16)
                plt.grid(True, linestyle="--", alpha=0.7)

                # Format x-axis correctly
                plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%d-%m-%Y"))  # Day-Month-Year
                plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())  # Auto-select reference dates
                plt.xticks(rotation=45)

                # Save plot as JPG
                output_filename = f"{station_name}_{city_name}_{pollutant}_Line_Plot.jpg"
                output_path = os.path.join(os.getcwd(), output_filename)
                plt.savefig(output_path, dpi=300, bbox_inches="tight")
                plt.close()

                print(f"Plot saved: {output_filename}")

In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageEnhance
import textwrap
import os

# **Step 1: Ask if User Wants to Contine**
choice = input("Do you want to create an infographic? (yes/no): ").strip().lower()
if choice != "yes":
    print("❌ Process exited. No infographic created.")
    # Instead of quit() or sys.exit(), use a simple return or
    # wrap the remaning code in an else block
    
else:
    # Step 2: Ask User for Inputs
    title_text = input("Enter the title: ")
    sub_title_text = input("Enter the subtitle: ")
    description_1 = input("Enter the first paragraph of description: ")
    description_2 = input("Enter the second paragraph of description (Optional, press Enter to skip): ")

    # Step 3: Set Standard Canvas Size
    canvas_width = 1920
    canvas_height = 1080
    canvas = Image.new("RGBA", (canvas_width, canvas_height), (255, 255, 255, 255))  # White base

    # Step 4: Load Background Image (Optional)
    background_path = input("Enter background image filename (Optional, press Enter to skip): ")

    if background_path and os.path.exists(background_path):
        background = Image.open(background_path).convert("RGBA")
        background = background.resize((canvas_width, canvas_height))
        background = background.filter(ImageFilter.GaussianBlur(radius=5))  # Slight blur effect
        canvas.paste(background, (0, 0))  # Paste only if available

    # Step 5: Procesing plots
    plot_paths = []  # Store plot file names
    for i in range(3):
        plot_file = input(f"Enter plot {i+1} filename (Optional, press Enter to skip): ")
        if plot_file and os.path.exists(plot_file):
            plot_paths.append(plot_file)

    
    plot_width = int(canvas_width * 0.45)  # 45% of width #  Plot Sizes and Positions
    plot_height = int(canvas_height * 0.25)  # 25% of height
    plot_positions = [
        (canvas_width // 2 + 20, int(canvas_height * 0.2)),
        (canvas_width // 2 + 20, int(canvas_height * 0.45)),
        (canvas_width // 2 + 20, int(canvas_height * 0.7)),
    ]

    for i, plot_path in enumerate(plot_paths):
        plot = Image.open(plot_path).convert("RGBA").resize((plot_width, plot_height))
        
        # **Enhance Tick Labels & Axis Labels**
        enhancer = ImageEnhance.Contrast(plot)
        plot = enhancer.enhance(2)  # Increase contrast to make text bolder
        plot = plot.filter(ImageFilter.EDGE_ENHANCE_MORE)  # Sharpen labels further

        # Remove White Background Without Affecting Axis
        datas = plot.getdata()
        new_data = []
        for item in datas:
            if item[:3] == (255, 255, 255):  # Detect pure white
                new_data.append((255, 255, 255, 100))  # Make it semi-transparent instead of fully transparent
            else:
                new_data.append(item)

        plot.putdata(new_data)  # Apply transparency
        canvas.paste(plot, plot_positions[i], plot)  # Paste with transparency

    # **Step 6: Add Text (Title, Subtitle, Description)**
    draw = ImageDraw.Draw(canvas)

    # Load Fonts (Ensure you have the TTF file 
    try:
        font_title = ImageFont.truetype("arial.ttf", 70)   # Title Font
        font_sub_title = ImageFont.truetype("arial.ttf", 50)  # Subtitle Font
        font_desc = ImageFont.truetype("arial.ttf", 35)   # Description Font
    except:
        font_title = ImageFont.load_default()
        font_sub_title = ImageFont.load_default()
        font_desc = ImageFont.load_default()

    #-------- Title (Centered at the top)-------
    title_position = (canvas_width // 4, 50)
    draw.text(title_position, title_text, fill="red", font=font_title)

    # Subtitle (Below Title)
    subtitle_position = (canvas_width // 4, 140)
    draw.text(subtitle_position, sub_title_text, fill="darkblue", font=font_sub_title)

    # **Step 7: Add Wrapped Description in Left Half**
    desc_position_1 = (50, int(canvas_height * 0.3))  # First Paragraph Position

    # Dynamically Adjust Second Paragraph Position
    wrapped_text_1 = textwrap.fill(description_1, width=50)
    text_height = draw.multiline_textbbox((0, 0), wrapped_text_1, font=font_desc)[3]  # Get height of first paragraph
    desc_position_2 = (50, desc_position_1[1] + text_height + 70)  # Move second paragraph below first

    # 1st para
    draw.multiline_text(desc_position_1, wrapped_text_1, fill="black", font=font_desc, spacing=10)

    # 2nd para
    if description_2:
        wrapped_text_2 = textwrap.fill(description_2, width=50)
        draw.multiline_text(desc_position_2, wrapped_text_2, fill="black", font=font_desc, spacing=10)

    canvas.save("final_infographic1.png")
    print("✅ Infographic created successfully as 'final_infographic.png'!")